## Initialization: Daily Data
I have pulled down USD, CHF, EUR, and GBP historical data from SIX.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='white', palette="tab10", font_scale=1.5, rc = {'figure.figsize':(12,6)})


import warnings
warnings.filterwarnings('ignore')
# a value has been set on a copy of a slice of a dataframe....

# Data directory
data_dir = Path() / 'data'
# Data source:
# https://www.six-group.com/en/products-services/the-swiss-stock-exchange/market-data/etp/etp-explorer/etp-detail.CH0454664001USD4.html#/

# Create dict of dataframes from data_dir
df_dict = {}
for file in data_dir.glob('*.csv'):
    # Semicolon delimited, and skip two lines for simplicity
    df_dict[file.stem] = pd.read_csv(file, sep=';', skiprows=2)

In [ ]:
GBP = df_dict.get('historical_GBP')
EUR = df_dict.get('historical_EUR')
USD = df_dict.get('historical_USD')
CHF = df_dict.get('historical_CHF')
markets = [GBP, EUR, USD, CHF]
markets_names = ['GBP', 'EUR', 'USD', 'CHF']
# concat dataframes and add name of symbol
master = pd.concat([GBP, EUR, USD, CHF], axis=1, keys=['GBP', 'EUR', 'USD', 'CHF'])

In [ ]:
master.head(20)

In [ ]:
for i, v in enumerate(markets):
    # Plot volume of each pair

    plot = sns.lineplot(x=v.index, y=v['Volume'], label = markets_names[i])
    plot.set_xlabel('Index / Time')
    plot.set_ylabel('Volume')
    plot.set_title('Most recent dates are towards the origin')
    # plot.invert_xaxis()

# from functions import sns_lineplot
# sns_lineplot(markets, markets_names)


In [ ]:
for i, v in enumerate(markets):
    # Plot volume of each pair with seaborn
    plot = sns.scatterplot(x=v.index, y=(v['Volume']), label = markets_names[i])
    plot.set_xlabel('Index / Volume')
    plot.set_ylabel('Volume')
    plot.set_title('Most recent dates are towards the origin')


#### Manipulation
Let's manipulate the original data so that the log plot of volume correctly shows zero days where they actually are.  
  
**Math nerd question: What problems are we going to encounter very shortly?**

In [ ]:
for df in markets:
    # replace 0 with 1
    df['Volume'] = df['Volume'].replace(0, 1)

for i, v in enumerate(markets):
    # Plot volume of each pair with seaborn
    plot = sns.scatterplot(x=v.index, y=np.log(v['Volume']), label = markets_names[i])
    # Add regression line for each pair
    # We can skip confidence interval
    plot = sns.regplot(x=v.index, y=np.log(v['Volume']), ci=False)
    plot.set_xlabel('Index / Time')
    plot.set_ylabel('Log (Volume)')
    plot.set_title('Most recent dates are towards the origin')

#### Takeaways
USD has the most volume, followed by CHF.  
This is in line with my expectation.  
The Euro has move volume than the Pound, but they both have some days with zero volume.  
I suspect this is reporting error.

Is there any correlation between the currencies?

In [ ]:
# Are the volumes correlated?
# plot correlation matrix
volume_only = pd.DataFrame()
for i, v in enumerate(markets):
    volume_only[markets_names[i]] = v['Volume']

In [ ]:
# plot correlation matrix
corr = volume_only.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(100, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=10)

In [ ]:
# Check correlation values
corr

The correlation between GBP and EUR volume is ~0.7; suspect that this is largely caused be the zero values.

 ##### Alternatives?
 What about turnover (volume * price) instead of only looking at share volume?
 
 Recall, though, that we are essentially comparing the same underlying across multiple markets, so one share is one share is one share.

 We could look at exchange rates at a minimum to see where there is more investigation warranted.

 I am confident that there are periodic arbitrage opportunities here, but that is well outside of the scope of this presentation.


## Initialization: Intra-Day Data

In [ ]:
# Data directory
data_dir = Path() / 'intraday_data'

# Create dict of dataframes from data_dir

for file in data_dir.glob('*.csv'):
    # Semicolon delimited, and skip two lines for simplicity
    # There is only one file in this folder.
    # https://www.six-group.com/en/products-services/the-swiss-stock-exchange/market-data/etp/etp-explorer/etp-detail.CH0454664001USD4.html#/
    df = pd.read_csv(file, sep=';', skiprows=2)

#### Trading hours
SIX trading hours are from 0900 to 1720 GMT +1, so let's take a look at the intra-day trades on the 24th of March.

In [ ]:
df
# 
# I'll assume this 17:27 is after hours trading
# It does seem correct that we have two trades immediately preceding the close at 17:19:58.73

In [ ]:
# VWAP = sum (price * volume) / sum (volume)
# Note I am reversing the dataframe here because we will use some forward looking functions
df = df[::-1]
# Vectorized for speed
df['VWAP'] = np.cumsum(df['Price'] * df['Volume']) / np.cumsum(df['Volume'])
print('Closing VWAP (not including the after hours trade) is: ', round(df['VWAP'].iloc[-2], 4))

## Hypothesis  
 If we look at Binance Spot price movement, we can likely see the price of ABTC following BTCUSDT in short order.  
 
 **Question: How quickly do prices follow?**

 First, though, let's estimate it whether these reported volumes are buys or sells.  
 Simple logic:
 
         DF['Pressure'] = BUY if next_price > current_price else SELL

*Note for later: we will need to shift these times to match GMT + 1*

In [ ]:
df['Delta'] = df['Price'].pct_change()
df.head()


In [ ]:
# Add column 'Pressure' if 'Delta' is positive
# df['Pressure'] = 'BUY' if df['Delta'] > 0 else 'SELL'     # truth error
df['Pressure'] = np.where(df['Delta'] > 0, 'BUY', 'SELL')
df['Pressure'] = np.where(df['Delta'] == 0, '0', df['Pressure'])
# Shift pressure backwards once so it corresponds to the correct volume row
df['Pressure'] = df['Pressure'].shift(-1)
print('Let us look at the first few rows and see what happens at market open:')
df.head(10)

**Some thoughts:**  
It looks like market maker trading activity at 09:11:47.  
This is probably the largest conjecture that I will make throughout the presentation.

In [ ]:
# List columns of df - something funny with trying to count the Time column
print(df.columns)
df.rename(columns={'        Time': 'Time'}, inplace=True)
print(df.columns)
df['Time'].value_counts().head(10)

In [ ]:
# print lines where 'Time' == 12:47:52.95
print(df.where(df['Time'] == '12:47:52.95').dropna())

Wow!  
There is a perfect example of slippage on this order.  
So... How much is it?


In [ ]:
turnover = 15.128 * 4009 + 15.102 * 593 + 15.004 * 1701
print("Dollar volume is ${0:.2f}".format(turnover))

A $95k transaction has moved the market by itself.  
But by how much?  
Not all of it was slippage, only about 1/3rd of the order was filled sub-par.

In [ ]:
avg_price = (15.128 * 4009 + 15.102 * 593 + 15.004 * 1701) / (4009 + 593 + 1701)
print("Average Price is ${0:.3f} compared to an initial fill at $15.128".format(avg_price))
# slippage_per_share = (15.128 - avg_price) / (4009 + 593 + 1701)
# slippage_per_share

In [ ]:
# total slippage =  expected * volume -  actual * volume
total = 15.128 * (4009 + 593 + 1701) - avg_price * (4009 + 593 + 1701)
print("Total USD slippage is: ${0:.2f} out of ${1:.2f}".format(total, turnover))
print("In percentage terms, this is {0:.5f}%".format(total / turnover * 100))
# Vectorize the following for speed
daily_dollars = sum (df['Price'].to_numpy() * df['Volume'].to_numpy() )
print('')
print("The daily turnover is ${0:.2f}".format(daily_dollars))

### Conclusion
Immediately the 0.237% figure caught my eye.  
I am curious what the standard slippage amount is for a $100k order.  
I suspect for the other currency pairs (EUR, GBP) this number would be higher.    
To better analyze the data, I needed to know the daily turnover.  

*This transaction was almost 1/5th of the daily volume for the instrument.*  

*0.237% for nearly 1/5th of the daily volume, however, seems quite reasonable.*

**Let's move on.**

It's time to compare the ABTC price to the price of the underlying.

## Get Binance minute data for 24 March 2022
I am using 5m candles here;  this will let us do some decent visual analysis inside the notebook without much hassle.

In [ ]:
# get binance minute data for 24 March 2022url = 'https://api.binance.com/api/v3/klines'
import requests
import json
import datetime as dt
url = 'https://api.binance.com/api/v3/klines'
symbol = 'BTCUSDT'
interval = '5m'
start = str(int(dt.datetime(2022,3,24).timestamp()*1000))
end = str(int(dt.datetime(2022,3,25).timestamp()*1000))
par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end}
data = pd.DataFrame(json.loads(requests.get(url, params= par).text))
#format columns name
data.columns = ['datetime', 'Open', 'High', 'Low', 'Close', 'Volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
data=data.astype(float)
# set index to datetime column
data.index = data['datetime']
# drop a few columns
data = data.drop(['datetime', 'close_time', 'qav', 'num_trades','taker_quote_vol', 'ignore'], axis=1)
data.head()

Something strange about the above - how does taker_base_vol not equal 1/2 * Volume?  
I wonder if Binance has so much volume that it sometimes pairs makers with makers?  
Again, beyond the scope of this presentation.  Does anyone know?

Don't forget - we are not in the same time zone.

In [ ]:
# Convert index to hh:mm:ss.ms
# Shift one hour to get GMT +1
data.index = pd.to_datetime(data.index, unit='ms').shift(1, freq='H')
# drop outside of time frame 0900 - 1730
data = data.loc[(data.index.hour >= 9) & (data.index.hour <= 17)]
data

Let's plot the Binance 5m data to get a visual of what happened on 24-March-2022.

In [ ]:
from math import pi
from bokeh.plotting import figure, show, output_notebook
output_notebook()

inc = data.Close > data.Open
dec = data.Open > data.Close
w = 5*60*1000 # half day in ms
data['date'] = pd.to_datetime(data.index, unit='ms')
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, width=1000, title = "Binance BTCUSDT 5m")
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(data.date, data.High, data.date, data.Low, color="black")
p.vbar(data.date[inc], w, data.Open[inc], data.Close[inc], fill_color="#207318", line_color="black")
p.vbar(data.date[dec], w, data.Open[dec], data.Close[dec], fill_color="#c40404", line_color="black")

show(p)


Let's double check our old SIX data

In [ ]:
df.head()

In [ ]:
# Remove miliseconds from Time column
df['date'] = df['Time'].str.split('.').str[0]

# Add 2022-03-24 to date column
df['date'] = '2022-03-24' + ' ' + df['date']
df['date'] = pd.to_datetime(df['date'])

#### Plot together
Let's take a look at the Binance Spot price and our ABTC price movements.

*Note: there is an issue since intraday data is sparse for ABTC.*


In [ ]:
import plotly.graph_objects as go
margins = dict(l=20, r=20, b=20, t=40)

fig = go.Figure(data=[go.Candlestick(x=data['date'],
                open=data['Open'], high=data['High'],
                low=data['Low'], close=data['Close'])
                     ])

fig.update_layout(xaxis_rangeslider_visible=False, title_text="Binance BTCUSDT 5m", margin=margins, height=390)
fig.show()

fig2 = go.Figure(data=[go.Line(x=df['Time'], y=df['Price']), ])
fig2.update_layout(xaxis_rangeslider_visible=False, title_text="SIX ABTC price movement", margin=margins, height=390)
fig2.show()

In [ ]:
df.sort_values(by='Volume', ascending=False).head()

## Dialing in
Let's take a look at the chart for the previously mentioned interesting points in time:   
12:47:52.95	  
15:47:17.32	

#### Hypothesis:
We will see movements at Binance for BTCUSDT preceed the price movements at SIX of ABTC.

Let's check the first time frame and compare to actual price movement.

In [ ]:
df[14:20]

What was going on on Binance?  
*Recall that Binance is on GMT*

![Before Noon](tv_images/noise.png)

The above looks largely inconsequential.  Unsure what to conclude here.  We can discuss if desired.

Let's check the second time period.

In [ ]:
df[20:25]

First let's look at the daily 1m bars.  Notice how this looks similar enough to our earlier 5m plot with bokeh:  

![Daily](tv_images/zoom_out.png)

Zooming in towards 15:47:17.33 where we had a huge price hike:

![Zoom In](tv_images/zoom_in.png)

#### Conclusion:
ABTC on SIX is liquid enough to follow the Binance spot price within one minute of a 0.77% price change.  
If we compared the different currency tickers, I suspect that we could find arbitrage opportunities here as well.



## What about live liquidity?
Let's find out how much liquidity is in the underlying and graph it.

In [ ]:
# 100 orders past top of book
r = requests.get("https://api.binance.com/api/v3/depth", params=dict(symbol="BTCUSDT", limit=100))
results = r.json()
frames = {side: pd.DataFrame(data=results[side], columns=["price", "quantity"],
                             dtype=float) for side in ["bids", "asks"]}
frames_list = [frames[side].assign(side=side) for side in frames]
data = pd.concat(frames_list, axis="index", ignore_index=True, sort=True)
price_summary = data.groupby("side").price.describe()
price_summary
data

In [ ]:
max = frames["bids"].price.max()
min = frames["asks"].price.min()
print('The current highest bid is ${0:.2f}'.format(max))
print('The current lowest ask is ${0:.2f}'.format(min))

We've looked at a bit too much data.  Let's start wrapping up.  
How much depth is there to the current liquidity?

In [ ]:
r = requests.get("https://api.binance.com/api/v3/ticker/bookTicker", params=dict(symbol="BTCUSDT"))
book_top = r.json()
name = book_top.pop("symbol")  # get symbol and also delete at the same time
s = pd.Series(book_top, name=name, dtype=float)
s


In [ ]:
# Order book scatter plot
fig, ax = plt.subplots()

ax.set_title(f"Current Order Book Depth for {name}")

sns.scatterplot(x="price", y="quantity", hue="side", data=data, ax=ax)

ax.set_xlabel("Price")
ax.set_ylabel("Count")

plt.show()

In [ ]:
# Order book histogram
fig, ax = plt.subplots()

ax.set_title(f"Current Order Book Depth for {name}")

sns.histplot(x="price", hue="side", binwidth=1, data=data, ax=ax)
sns.rugplot(x="price", hue="side", data=data, ax=ax)

plt.show()

In [ ]:
# Order book weighted histogram - now we are getting somewhere
fig, ax = plt.subplots()

ax.set_title(f"Current Order Book Depth for {name}")

sns.histplot(x="price", weights="quantity", hue="side", binwidth=1, data=data, ax=ax)
sns.scatterplot(x="price", y="quantity", hue="side", data=data, ax=ax)

ax.set_xlabel("Price")
ax.set_ylabel("Quantity")

plt.show()

In [ ]:
# Order book depth chart

fig, ax = plt.subplots()

ax.set_title(f"Current Order Book Depth for {name}")
sns.ecdfplot(x="price", weights="quantity", stat="count", complementary=True, data=frames["bids"], ax=ax)
sns.ecdfplot(x="price", weights="quantity", stat="count", data=frames["asks"], ax=ax)
sns.scatterplot(x="price", y="quantity", hue="side", data=data, ax=ax)

ax.set_xlabel("Price")
ax.set_ylabel("Quantity")

plt.show()

Finale: how much liquidity is there at this very moment with NO SLIPPAGE?

In [ ]:
import time
from IPython.display import clear_output
counter = 0

while counter <= 20:
    now = time.time()
    r = requests.get("https://api.binance.com/api/v3/ticker/bookTicker", params=dict(symbol="BTCUSDT"))
    book_top = r.json()
    name = book_top.pop("symbol")  # get symbol and also delete at the same time
    s = pd.Series(book_top, name=name, dtype=float)
    buyer_liquidity = s.get("askPrice") * s.get("askQty")
    seller_liquidity = s.get("bidPrice") * s.get("bidQty")
    delay = time.time() - now
    print('Request and response delay: {0:.2f} seconds'.format(delay))
    print(f"Buyer liquidity: ${round(buyer_liquidity, 2)}, BTC quantity: {(s.get('askQty'))}")
    print(f"Seller liquidity: ${round(seller_liquidity, 2)}, BTC quantity: {(s.get('bidQty'))}")
    time.sleep(1)
    counter += 1
    clear_output(wait=True)

## Final Thoughts
It would be relatively easy to build a tool that finds current liquidity within a allowable slippage percent for the underlying, but BTC on Binance still has some issues, the main one being:  
* **Order book spoofing is rampant.**  

This could be mitigated by some combination of order types, such as Limit + Fill or Kill.  
To the best of my knowledge, frontrunning a la Hedge Funds ("Flash Boys" - Michael Douglas) does not happen by other market participants.  This is beneficial for liquidity takers. 

There is some lag time between when the price moves on Binance and the price moves on SIX.  
  
I am quite impressed that 1/5th of the daily volume on ABTC only incurred 0.25% slippage!  

____

Final questions I have for you:  
How often do you need to rebalance the ETP?  
How often do you issue new shares?

This concludes my presentation on crypto ETP and underlying liquidity.